# Interpolating Points Along Lines

This notebook shows how to take a line layer and generate equally spaced points along each line feature using GeoPandas.

We use `shapely` library for performing the interpolation. 

![](line_interpolation.png)

In [1]:
import geopandas as gpd
from shapely.geometry import MultiPoint

We read a sample line layer containing 2 line features.

In [2]:
gdf = gpd.read_file('line.shp')

In [3]:
gdf

,FID,geometry
0,0,"LINESTRING (503423.417 492226.902, 545830.203 ..."
1,1,"LINESTRING (508389.969 484415.756, 522917.195 ..."


Check and verify that the CRS is a projected CRS. The layer must be in a Projected CRS and the distance units must be in the same unit of the CRS. This code will **NOT** work on layers which are in EPSG:4326 CRS. Re-project your data to a projected CRS before using.

In [4]:
gdf.crs

<Projected CRS: EPSG:27700>
Name: OSGB36 / British National Grid
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: United Kingdom (UK) - offshore to boundary of UKCS within 49°45'N to 61°N and 9°W to 2°E; onshore Great Britain (England, Wales and Scotland). Isle of Man onshore.
- bounds: (-9.0, 49.75, 2.01, 61.01)
Coordinate Operation:
- name: British National Grid
- method: Transverse Mercator
Datum: Ordnance Survey of Great Britain 1936
- Ellipsoid: Airy 1830
- Prime Meridian: Greenwich

Write a function that will be applied on each feature. The code of this function is adapted from [this post](https://stackoverflow.com/questions/34906124/interpolating-every-x-distance-along-multiline-in-shapely/35025274#35025274).

In [5]:
# distance in meters
distance = 1000

def get_points(row):
    geometry = row.geometry
    num_vert = int(round(geometry.length / distance))

    if num_vert == 0:
        num_vert = 1
    
    points = []
    for n in range(num_vert + 1):
        point = geometry.interpolate(n/num_vert, normalized=True)
        points.append(point)
    return MultiPoint(points)


Apply the function on each line in the layer and save to a new column

In [6]:
gdf['points'] = gdf.apply(get_points, axis=1)

Set the new column as the geometry and remove the old one

In [7]:
gdf = gdf.set_geometry('points')
gdf = gdf.drop(columns=['geometry'])
gdf

,FID,points
0,0,"MULTIPOINT (503423.417 492226.902, 504345.304 ..."
1,1,"MULTIPOINT (508389.969 484415.756, 509248.075 ..."


Each line has many points associated with it as a MultiPoint feature. Convert them to individual features

In [8]:
gdf_final = gdf.explode()
gdf_final

FID                         points
0 0     0  POINT (503423.417 492226.902)
  1     0  POINT (504345.304 492609.275)
  2     0  POINT (505267.190 492991.649)
  3     0  POINT (506189.077 493374.022)
  4     0  POINT (507110.963 493756.396)
...   ...                            ...
1 45    1  POINT (547131.143 502101.348)
  46    1  POINT (547654.000 502960.327)
  47    1  POINT (548176.856 503819.306)
  48    1  POINT (548699.713 504678.284)
  49    1  POINT (549222.570 505537.263)

[97 rows x 2 columns]

Save the output as a shapefile.

In [99]:
gdf_final.to_file('interpolated.shp')